Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Azure Machine Learning Compute Instance

* Initialize Workspace
* Introduction to ComputeInstance
* Additional operations to perform on ComputeInstance

## Get a Workspace

Get a workspace object from a `config.json` file

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Introduction to ComputeInstance


Azure Machine Learning compute instance is a fully-managed cloud-based workstation optimized for your machine learning development environment. It is created **within your workspace region**.

For more information on ComputeInstance, please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-instance)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

### Create ComputeInstance
First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since ComputeInstance is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D3_V2') is supported.

In [ ]:
from azureml.core.compute import ComputeTarget, ComputeInstance

ComputeInstance.supported_vmsizes(workspace = ws)
# ComputeInstance.supported_vmsizes(workspace = ws, location='eastus')

In [ ]:
import datetime
import time

from azureml.core.compute import ComputeTarget, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your instance
# Compute instance name should be unique across the azure region
compute_name = "ci{}".format(ws._workspace_id)[:10]

# Verify that instance does not exist already
try:
    instance = ComputeInstance(workspace=ws, name=compute_name)
    print('Found existing instance, use it.')
except ComputeTargetException:
    compute_config = ComputeInstance.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        ssh_public_access=False,
        # vnet_resourcegroup_name='<my-resource-group>',
        # vnet_name='<my-vnet-name>',
        # subnet_name='default',
        # admin_user_ssh_public_key='<my-sshkey>'
    )
    instance = ComputeInstance.create(ws, compute_name, compute_config)
    instance.wait_for_completion(show_output=True)

### Additional operations to perform on ComputeInstance

You can perform more operations on ComputeInstance such as get status, change the state or deleting the compute.

In [ ]:
# get_status() gets the latest status of the ComputeInstance target
instance.get_status()

In [ ]:
# stop() is used to stop the ComputeInstance
# Stopping ComputeInstance will stop the billing meter and persist the state on the disk.
# Available Quota will not be changed with this operation.
instance.stop(wait_for_completion=True, show_output=True)

In [ ]:
# start() is used to start the ComputeInstance if it is in stopped state
instance.start(wait_for_completion=True, show_output=True)

In [ ]:
# restart() is used to restart the ComputeInstance
instance.restart(wait_for_completion=True, show_output=True)

In [ ]:
# delete() is used to delete the ComputeInstance target. Useful if you want to re-use the compute name 
# instance.delete(wait_for_completion=True, show_output=True)